In [ ]:
# %load_ext autoreload
# %autoreload 2

import os,sys,pickle
import h5py  
import numpy as np
import math,cmath
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['lines.markersize'] = 12
mpl.rcParams['errorbar.capsize'] = 12
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['legend.fontsize'] = 24
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['figure.titlesize'] = 30
mpl.rcParams.update({"axes.grid" : True})

if not os.path.abspath('../util') in sys.path:
    sys.path.append(os.path.abspath('../util'))
import util as yu

ensList=['cA48','cB64','cC80','cD96']
ensList=['cB64','cC80','cD96']
ens2ensemble={'cA48':'cA2.09.48','cB64':'cB211.072.64','cC80':'cC211.060.80','cD96':'cD211.054.96'}
ens2LAT_T={'cA48':48*2,'cB64':64*2,'cC80':80*2,'cD96':96*2}
ens2afm={'cA48':1,'cB64':0.07957,'cC80':0.06821,'cD96':0.05692}
ens2MUL={'cA48':0.0009,'cB64':0.00072,'cC80':0.0006,'cD96':0.00054}
ens2ZPbyZS={'cA48':1,'cB64':0.79018,'cC80':0.82308,'cD96':0.85095}
ens2ZP={'cA48':1,'cB64':0.4746,'cC80':0.4771,'cD96':0.4871}
ens2mNGeV={'cA48':1,'cB64':0.9464,'cC80':0.9436,'cD96':0.9414}

path_data='/p/project/pines/li47/code/projectData/discNJN/temp/data.h5'

# %matplotlib widget

In [ ]:
data={}
with h5py.File(path_data) as f:
    for ens in f.keys():
        data[ens]={}
        for diag in f[ens].keys():
            data[ens][diag]={}
            for ky in f[ens][diag].keys():
                data[ens][diag][ky]=f[ens][diag][ky][:]
            print(ens,diag)

In [ ]:
tf=14

with h5py.File('/p/project/pines/li47/code/projectData/discNJN/others/cB64/thrp-conn.h5') as f:
    ky='dt'+str(tf)+'_twop'
    t=[f[ky][cfg][()] for cfg in f[ky].keys()]
    t=np.mean(t)
    print(t)
    
t=data['cB64']['N']['N1,N1'][:,tf,3,0]+data['cB64']['N']['N1,N1'][:,tf,3,5]
np.mean(t)/2

In [ ]:
tfList_all=range(6,26+1,2)
tfList=tfList_all
def getdat(ens,insert,case):
    moms=data[ens]['N']['mvec']
    momDic={}
    for i,mom in enumerate(moms):
        momDic[tuple(mom)]=i
        
    p0=[0,0,0]; pf1=[0,0,0]
    ind=momDic[tuple(pf1+p0+p0+pf1+p0)]
    t_2pta=(
        data[ens]['N']['N1,N1'][:,:,ind,0] + data[ens]['N']['N1,N1'][:,:,ind,5] +
        data[ens]['N']['N2,N2'][:,:,ind,0] + data[ens]['N']['N2,N2'][:,:,ind,5]
    )/4
    
    Ncfg=len(data[ens]['N']['N1,N1'])

    p0=[0,0,0]; pf1=[0,0,0]
    ind=momDic[tuple(pf1+p0+p0+pf1+p0)] 
    t_2ptb=(
        data[ens]['N']['N1,N1'][:,:,ind,0] + data[ens]['N']['N1,N1'][:,:,ind,5] +
        data[ens]['N']['N2,N2'][:,:,ind,0] + data[ens]['N']['N2,N2'][:,:,ind,5]
    )/4
    
    moms=data[ens]['N-j']['mvec']
    momDic={}
    for i,mom in enumerate(moms):
       momDic[tuple(mom)]=i

    
    momMap_p2pN=[momDic[tuple(-np.array(mom))] for mom in moms]
    momMap_src2sink=[momDic[tuple(list(mom[9:12])+[0,0,0]+list(-np.array(mom[6:9]))+list(mom[0:3])+[0,0,0])] for mom in moms]

    (Gc,pol,iso,prefactor,pc)=insert

    sgn_fla= -1 if iso in ['j-'] else 1
    sgn_Jdr= -1 if Gc in [1,2,3,5,6,7,8,10,11,12] else 1
    sgn_PJP= -1 if Gc in [1,2,3,5,9,13,14,15] else 1
    sgn_TJT= -1 if Gc in [4,5,6,7,8,13,14,15] else 1
    # sgn_Jrt= -1 if Gc in [3,6,7,8] else 1 if Gc in [0,1,4,5,9] else None

        
    z3=[0,0,0]
    momMap=momDic[tuple(z3+z3+pc+pc+z3)]
    t_3pt={}
    for tf in tfList:
       t=np.array([[
       (data[ens]['N-j']['N1,'+iso+',N1_deltat_'+str(tf)][:,:,:,Gc,0]+pol*
              data[ens]['N-j']['N1,'+iso+',N1_deltat_'+str(tf)][:,:,:,Gc,5])/2,
       (data[ens]['N-j']['N2,'+iso+',N2_deltat_'+str(tf)][:,:,:,Gc,0]+pol*
              data[ens]['N-j']['N2,'+iso+',N2_deltat_'+str(tf)][:,:,:,Gc,5])/2
       ],
       [
       (data[ens]['N-jbw']['N1,'+iso+',N1_deltat_'+str(tf)][:,:,:,Gc,10]+pol*
              data[ens]['N-jbw']['N1,'+iso+',N1_deltat_'+str(tf)][:,:,:,Gc,15])/2,
       (data[ens]['N-jbw']['N2,'+iso+',N2_deltat_'+str(tf)][:,:,:,Gc,10]+pol*
              data[ens]['N-jbw']['N2,'+iso+',N2_deltat_'+str(tf)][:,:,:,Gc,15])/2
       ]
       ])

       # c.c.
       tt=sgn_Jdr*np.conj(t[:,:,:,:,momMap_src2sink])
       t=(t+tt)/2

       # P*fla
       tt=sgn_PJP*sgn_fla*np.array([[t[0,1],t[0,0]],[t[1,1],t[1,0]]]); tt=tt[:,:,:,:,momMap_p2pN]
       t=(t+tt)/2

    #    # Rot
    #    tt=pol*t[:,:,:,:,momMap_p2pN]*sgn_Jrt
    #    t=(t+tt)/2

       # T*fla
       tt=(-1)*sgn_TJT*sgn_fla*np.array([[t[1,1],t[1,0]],[t[0,1],t[0,0]]])
       t=(t+tt)/2 
       
       t1=prefactor*t[0,0,:,:,momMap]
       t2=prefactor*data[ens]['NJN']['gS_j-_deltat_'+str(tf)]

       t_3pt[tf]={'conn':t2,'disc':t1,'conn+disc':t1+t2}[case]

    t=np.array([
        np.array([t_2pta[cfg],t_2ptb[cfg]]+[np.array([t_3pt[tf][cfg][1:tf] for tf in tfList],dtype=object)],dtype=object)
    for cfg in range(Ncfg)], dtype=object)
    
    return t
        
def run(insert):
    for ens in ensList:
        fig, axs = yu.getFigAxs(1,3,Lrow=4*1.5,Lcol=5*1.5)
        fig.suptitle(ens)
        yu.addColHeader(axs,['conn','disc','conn+disc'])
        for icol,case in enumerate(['conn','disc','conn+disc']):
            hbarc = 1/197.3
            LAT_T=ens2LAT_T[ens]
            mu=ens2MUL[ens]
            ZPbyZS=ens2ZPbyZS[ens]
            ZP=ens2ZP[ens]
            mNGeV=ens2mNGeV[ens]
            afm=ens2afm[ens]; aInvMeV=1/afm/hbarc
            mN=mNGeV*afm*1000*hbarc
            
            q=2*np.pi/(LAT_T/2)
            xE=np.sqrt(mN**2 + q**2)
            xC=np.sqrt(2*mN**2/(xE*(xE+mN)))
        #     print(afm,aInvMeV)
        #     print(mu,aInvMeV,ZP,ZPbyZS,mu/ZP*aInvMeV)

            (Gc,pol,iso,prefactor,pc)=insert
            
            if (Gc,pol,iso) == (5,-1,'j-'):
                factor=-(mu/ZPbyZS)/(xC*q/(2*mN))*aInvMeV # for mqG5
                # factor=-(ZP/ZPbyZS)/(xC*q/(2*mN)) # for G5
            elif (Gc,pol,iso) == (0,1,'j-'):
                factor={'cB64':0.47,'cC80':0.487,'cD96':0.493}[ens]


            dat=getdat(ens,insert,case)
            
            def tFunc(dat):
                t=np.mean(dat,axis=0)

                def getRes(i_tf):
                    tf=tfList[i_tf]

                    nom=t[2][i_tf]
                    denom=np.sqrt(np.real(t[0][tf])*np.real(t[1][tf]) *\
                    np.real(t[0][1:tf])[::-1]/np.real(t[0][1:tf]) *\
                    np.real(t[1][1:tf])/np.real(t[1][1:tf])[::-1]
                        )
            #      return np.real(nom)
                    return np.real(factor*nom/denom)
                    
                t=np.array([getRes(ind_tf) for ind_tf in range(len(tfList))],dtype=object)
                return t
            
        #     tFunc(dat)
        #     break
            (mean,cov,err) = yu.jackknife(dat,tFunc)
            colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
            fmtDic={10:'s',12:'d',14:'o'}

            irow=0
            for i_tf in range(len(tfList)):
                tf=tfList[i_tf]
                tMean=mean[i_tf];tErr=err[i_tf]
                axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,\
                    fmt='s',mfc='white')
            axs[irow,icol].set_ylim([-0.5,1.5])
            
tfList=range(8,18+1,2)
run((0,1,'j-',1,[0,0,0]))

In [ ]:
tfList_all=range(6,26+1,2)
tfList=tfList_all
def getdat(ens,insert):
    moms=data[ens]['N']['mvec']
    momDic={}
    for i,mom in enumerate(moms):
        momDic[tuple(mom)]=i
        
    p0=[0,0,0]; pf1=[0,0,0]
    ind=momDic[tuple(pf1+p0+p0+pf1+p0)]
    t_2pta=(
        data[ens]['N']['N1,N1'][:,:,ind,0] + data[ens]['N']['N1,N1'][:,:,ind,5] +
        data[ens]['N']['N2,N2'][:,:,ind,0] + data[ens]['N']['N2,N2'][:,:,ind,5]
    )/4
    
    Ncfg=len(data[ens]['N']['N1,N1'])

    p0=[0,0,0]; pf1=[0,0,1]
    ind=momDic[tuple(pf1+p0+p0+pf1+p0)] 
    t_2ptb=(
        data[ens]['N']['N1,N1'][:,:,ind,0] + data[ens]['N']['N1,N1'][:,:,ind,5] +
        data[ens]['N']['N2,N2'][:,:,ind,0] + data[ens]['N']['N2,N2'][:,:,ind,5]
    )/4
    
    moms=data[ens]['N-j']['mvec']
    momDic={}
    for i,mom in enumerate(moms):
       momDic[tuple(mom)]=i

    
    momMap_p2pN=[momDic[tuple(-np.array(mom))] for mom in moms]
    momMap_src2sink=[momDic[tuple(list(mom[9:12])+[0,0,0]+list(-np.array(mom[6:9]))+list(mom[0:3])+[0,0,0])] for mom in moms]

    (Gc,pol,iso,prefactor,pc)=insert

    sgn_fla= -1 if iso in ['j-'] else 1
    sgn_Jdr= -1 if Gc in [1,2,3,5,6,7,8,10,11,12] else 1
    sgn_PJP= -1 if Gc in [1,2,3,5,9,13,14,15] else 1
    sgn_TJT= -1 if Gc in [4,5,6,7,8,13,14,15] else 1
    # sgn_Jrt= -1 if Gc in [3,6,7,8] else 1 if Gc in [0,1,4,5,9] else None

        
    z3=[0,0,0]
    momMap=momDic[tuple(z3+z3+pc+pc+z3)]
    t_3pt={}
    for tf in tfList:
       t=np.array([[
       (data[ens]['N-j']['N1,'+iso+',N1_deltat_'+str(tf)][:,:,:,Gc,0]+pol*
              data[ens]['N-j']['N1,'+iso+',N1_deltat_'+str(tf)][:,:,:,Gc,5])/2,
       (data[ens]['N-j']['N2,'+iso+',N2_deltat_'+str(tf)][:,:,:,Gc,0]+pol*
              data[ens]['N-j']['N2,'+iso+',N2_deltat_'+str(tf)][:,:,:,Gc,5])/2
       ],
       [
       (data[ens]['N-jbw']['N1,'+iso+',N1_deltat_'+str(tf)][:,:,:,Gc,10]+pol*
              data[ens]['N-jbw']['N1,'+iso+',N1_deltat_'+str(tf)][:,:,:,Gc,15])/2,
       (data[ens]['N-jbw']['N2,'+iso+',N2_deltat_'+str(tf)][:,:,:,Gc,10]+pol*
              data[ens]['N-jbw']['N2,'+iso+',N2_deltat_'+str(tf)][:,:,:,Gc,15])/2
       ]
       ])

       # c.c.
       tt=sgn_Jdr*np.conj(t[:,:,:,:,momMap_src2sink])
       t=(t+tt)/2

       # P*fla
       tt=sgn_PJP*sgn_fla*np.array([[t[0,1],t[0,0]],[t[1,1],t[1,0]]]); tt=tt[:,:,:,:,momMap_p2pN]
       t=(t+tt)/2

    #    # Rot
    #    tt=pol*t[:,:,:,:,momMap_p2pN]*sgn_Jrt
    #    t=(t+tt)/2

       # T*fla
       tt=(-1)*sgn_TJT*sgn_fla*np.array([[t[1,1],t[1,0]],[t[0,1],t[0,0]]])
       t=(t+tt)/2 

       t_3pt[tf]=prefactor*t[0,0,:,:,momMap]

    t=np.array([
        np.array([t_2pta[cfg],t_2ptb[cfg]]+[np.array([t_3pt[tf][cfg][1:tf] for tf in tfList],dtype=object)],dtype=object)
    for cfg in range(Ncfg)], dtype=object)
    
    return t
        
def run(insert):
    for ens in ensList:
        hbarc = 1/197.3
        LAT_T=ens2LAT_T[ens]
        mu=ens2MUL[ens]
        ZPbyZS=ens2ZPbyZS[ens]
        ZP=ens2ZP[ens]
        mNGeV=ens2mNGeV[ens]
        afm=ens2afm[ens]; aInvMeV=1/afm/hbarc
        mN=mNGeV*afm*1000*hbarc
        
        q=2*np.pi/(LAT_T/2)
        xE=np.sqrt(mN**2 + q**2)
        xC=np.sqrt(2*mN**2/(xE*(xE+mN)))
    #     print(afm,aInvMeV)
    #     print(mu,aInvMeV,ZP,ZPbyZS,mu/ZP*aInvMeV)

        (Gc,pol,iso,prefactor,pc)=insert
        
        if (Gc,pol,iso) == (5,-1,'j-'):
            factor=-(mu/ZPbyZS)/(xC*q/(2*mN))*aInvMeV # for mqG5
            # factor=-(ZP/ZPbyZS)/(xC*q/(2*mN)) # for G5
        else:
            factor=1

        fig, axs = yu.getFigAxs(1,1,Lrow=4*1.5,Lcol=5*1.5)
        fig.suptitle(ens)
        dat=getdat(ens,insert)
        
        def tFunc(dat):
            t=np.mean(dat,axis=0)

            def getRes(i_tf):
                tf=tfList[i_tf]

                nom=t[2][i_tf]
                denom=np.sqrt(np.real(t[0][tf])*np.real(t[1][tf]) *\
                np.real(t[0][1:tf])[::-1]/np.real(t[0][1:tf]) *\
                np.real(t[1][1:tf])/np.real(t[1][1:tf])[::-1]
                    )
        #      return np.real(nom)
                return np.real(factor*nom/denom)
                
            t=np.array([getRes(ind_tf) for ind_tf in range(len(tfList))],dtype=object)
            return t
        
    #     tFunc(dat)
    #     break
        (mean,cov,err) = yu.jackknife(dat,tFunc)
        colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
        fmtDic={10:'s',12:'d',14:'o'}

        irow=0; icol=0
        for i_tf in range(len(tfList)):
            tf=tfList[i_tf]
            tMean=mean[i_tf];tErr=err[i_tf]
            axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,\
                fmt='s',mfc='white')
            
tfList=range(6,18+1,2)
run((0,1,'j-',1,[0,0,0]))